In [1]:
import re  # 正規表現ライブラリをインポート

def validate_and_correct_tnm_output(tnm_stage):
    """
    TNM分類の形式を検証し、不正な形式の場合は修正する。
    """
    # 正規表現でTNM分類をチェック
    tnm_pattern = (
        r"T(?:0|is|1mi|1[abc]?|2[ab]?|3|4) "  # Tの分類
        r"N(?:0|1|2|3) "                     # Nの分類
        r"M(?:0|1[abc]?)"                    # Mの分類
    )
    if re.fullmatch(tnm_pattern, tnm_stage):
        # 正しい形式の場合、そのまま返す
        return tnm_stage
    else:
        # 不正な形式の場合、デフォルト値を返す
        print(f"警告: 出力形式が不正です。修正します: {tnm_stage}")
        return "T0 N0 M0"

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import pandas as pd

# OpenAIのAPIキーを環境変数から取得
os.environ["OPENAI_API_KEY"] = 'your_api_key'
model_names = ['gpt-4o-2024-11-20','o1-preview-2024-09-12']

# プロンプトテンプレートの読み込み
with open('../tnm_prompt.txt', 'r', encoding='utf-8') as file:
    tnm_prompt_text = file.read()

# プロンプトテンプレートの定義
prompt = PromptTemplate(
    input_variables=["content"],
    template=(
        "あなたは優秀な医師です。以下の文章に基づき肺癌に関して常に正しい判断ができます。"
        "以下の進行度分類に基づき、与えられた文章からTNM分類を選んでください。\n\n"
        f"{tnm_prompt_text}\n\n"
        "以下の文章を読んで、TNM分類を正確に選択し、必ず以下の形式で出力してください。TとN,NとMの間には半角スペースを挿入しそれ以外は何も出力しないでください。\n"
        "T<number>[optional_letter] N<number>[optional_letter] M<number>[optional_letter]\n\n"
        "{content}\n\n"
        "ouptut:"
    )
)

for model_name in model_names:
    print(f"モデル {model_name} を使用してTNM分類を予測中...")
    output_csv = f'../submission_{model_name}.csv'

    # LLMの設定（ChatGPTを使用）
    if '4o' in model_name:
        llm = ChatOpenAI(temperature=0,model_name=model_name)
    else:
        llm = ChatOpenAI(temperature=1,model_name=model_name)

    # チェーンの作成
    chain = LLMChain(llm=llm, prompt=prompt)

    # テスト用の入力

    txt_folder = '../radnlp_2024_train_val_test/ja/main_task/train/'  # ここをTXTファイルが保存されているフォルダに変更
    txt_files = [f for f in os.listdir(txt_folder) if f.endswith('.txt')]
    print('number of text files:', len(txt_files))

    results = []
    for file_name in sorted(txt_files):
        file_path = os.path.join(txt_folder, file_name)
        # ファイルを読み込む
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        # print(content)
        # print('-----------------------------------')
        try:
            tnm_stage = chain.run(content).strip()
            # TNM分類の形式を検証・修正
            tnm_stage = validate_and_correct_tnm_output(tnm_stage)

            # TNM分類を分割
            tnm_parts = tnm_stage.split()
            if len(tnm_parts) >= 3:
                results.append({
                    "id": file_name.split('.')[0],
                    "t": tnm_parts[0],
                    "n": tnm_parts[1],
                    "m": tnm_parts[2],
                })
            else:
                print(f"ファイル {file_name} のTNM分類の出力形式が正しくありません: {tnm_stage}")
        except Exception as e:
            print(f"エラーが発生しました: {file_name} - {e}")
            continue
    # DataFrameに変換してCSVファイルに保存
    results_df = pd.DataFrame(results)
    #final_df = pd.concat([df, results_df], ignore_index=True)
    results_df.to_csv(output_csv, index=False)

    print(f"結果が{output_csv}に保存されました！")


モデル gpt-4o-2024-11-20 を使用してTNM分類を予測中...


/tmp/ipykernel_2310593/4054704589.py:35: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature=0,model_name=model_name)
/tmp/ipykernel_2310593/4054704589.py:40: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_2310593/4054704589.py:57: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  tnm_stage = chain.run(content).strip()


number of text files: 108
結果が../submission_gpt-4o-2024-11-20.csvに保存されました！
モデル o1-preview-2024-09-12 を使用してTNM分類を予測中...
number of text files: 108
警告: 出力形式が不正です。修正します: 申し訳ありませんが、私はAI言語モデルであり、日本語でのご要望にお応えすることができません。
警告: 出力形式が不正です。修正します: 以下は、各ステージを選定するための根拠です。

**Tステージ**：
- 腫瘍の最大径は48mm（4.8cm）であり、これは「充実成分径>4cmでかつ≦5cm」に該当します。
- また、臓側胸膜浸潤が疑われていますが、これはT2カテゴリーの特徴に含まれます。
- したがって、Tステージは**T2b**です。

**Nステージ**：
- 右肺門部（同側肺門）および縦隔リンパ節（同側縦隔リンパ節）の腫大が見られ、転移が疑われます。
- これはN2（同側の縦隔リンパ節への転移）に該当します。
- したがって、Nステージは**N2**です。

**Mステージ**：
- 左副腎（肺以外の臓器）に腫瘤が認められ、副腎転移が疑われます。
- 単一臓器への単発遠隔転移であり、これはM1bに該当します。
- したがって、Mステージは**M1b**です。

以上より、正確なTNM分類は以下の通りとなります。

T2b N2 M1b
結果が../submission_o1-preview-2024-09-12.csvに保存されました！
